# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import shutil

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

from azureml.core.model import Model
from azureml.widgets import RunDetails
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice


# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

NameError: name 'azureml' is not defined

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [ ]:
#create an experiment for hyperdrive run
ws = Workspace.from_config()
experiment_name = 'Hyperdrive-California-housing-price-projection'

experiment=Experiment(ws, experiment_name)

In [ ]:
#create compute cluster
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',vm_priority='lowpriority',max_nodes=4)
compute_target = ComputeTarget.create(ws, "compute-hyper", compute_config)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [ ]:
os.makedirs('./outputs', exist_ok = True)

In [ ]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(slack_factor=0.1, evaluation_interval=1)

#TODO: Create the different params that you will be using during training
ps = RandomParameterSampling({
    "--max_depth": choice(64),
    "--min_samples_split": choice(50, 1000), 
    "--min_samples_leaf": choice(10),
    })
#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(source_directory='.', compute_target = compute_target, vm_priority = 'lowpriority', entry_script='train.py')

hyperdrive_run_config = HyperDriveConfig(
    hyperparameter_sampling=ps,
    primary_metric_name='r2_score', 
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs = 20,
    max_concurrent_runs = 3,
    policy= early_termination_policy,
    estimator = estimator)

In [ ]:
#TODO: Submit your experiment
from azureml.train.hyperdrive import HyperDriveRun
hyperdrive_run = experiment.submit(hyperdrive_run_config)


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion()

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
import joblib
# Get your best run and save the model from that run.
best_hyperdrive_run = hyperdrive_run.get_best_run_by_primary_metric()
best_hyperdrive_run.get_metrics()

In [ ]:
#TODO: Save the best model
#Model saved through train.py 

In [ ]:
os.getcwd()

In [ ]:
best_hyperdrive_run.get_file_names()

In [ ]:
best_hyperdrive_run.download_file('outputs/hyper-model.joblib',output_file_path = './outputs')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
#The outputs folder somehow doesn't show up in the files directory
#joblib.dump(best_hyperdrive_run, 'model.joblib')
best_hyperdrive_model = best_hyperdrive_run.register_model(model_path = 'outputs/hyper-model.joblib', model_name = "DT_model")

In [ ]:
os.listdir()

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
from azureml.core.model import Model
print(os.getenv('AZUREML_MODEL_DIR'))
print(os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'hyper-model.joblib'))
first_model = Model(ws, name = "DT_model")
print(first_model.get_model_path("./outputs/hyper-model.joblib"))

In [ ]:
#create an inference configuration
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

#env = Environment.from_conda_specification("myenv", "./environment.yml")
env = Environment.get(workspace = ws, name = "AzureML-AutoML")

for pip_package in ["pandas","scikit-learn","joblib"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='score.py',
                                    environment=env)

In [26]:
from azureml.core.webservice import AciWebservice, AksWebservice, LocalWebservice
#deployment_config = AksWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
from azureml.core.webservice import LocalWebservice, Webservice
from azureml.core.model import InferenceConfig, Model

#deployment_config = LocalWebservice.deploy_configuration(port = 9000)
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, enable_app_insights = True)
first_model = Model(ws, name = "DT_model")
service = Model.deploy(ws, "hyperservice", [first_model], inference_config, deployment_config, overwrite = True)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.get_logs())

In [ ]:
Model.get_model_path("DT_model")

In [ ]:
import json
data = """{'data':
[
{'longitude': -1.2025339157924844,
 'latitude': 0.798943230972673,
 'housing_median_age': 0.3468139212594067,
 'total_rooms': 0.42718693585622175,
 'total_bedrooms': 0.5651250878784122,
 'population': 0.43863527266031216,
 'households': 0.6842099529470617,
 'median_income': 0.5712999263108347,
 'ocean_proximity_<1H OCEAN': 1.12329372733702,
 'ocean_proximity_INLAND': -0.6827132915603219,
 'ocean_proximity_ISLAND': -0.015644874280329252,
 'ocean_proximity_NEAR BAY': -0.3535241910124512,
 'ocean_proximity_NEAR OCEAN': -0.3841861385422699}
 ]
 }"""



In [ ]:
import requests

headers = {'Content-type': 'application/json'}

response = requests.post(service.scoring_uri, data, headers = headers)

In [ ]:
print(response.text)

In [ ]:
service.get_logs()

In [ ]:
#service.delete()